In [1]:
import nltk
import pandas as pd
import random
import numpy
import string
import math

from numpy import array
from nltk.corpus import stopwords
from collections import Counter
from scipy.sparse import csr_matrix
from nltk.stem.snowball import FrenchStemmer
from nltk.tokenize import word_tokenize

from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel
from gensim import corpora, models, utils
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim.models import LsiModel
from gensim.corpora import Dictionary

from nltk import ngrams


C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#TODO: Lemmatize correctly, this lemmatizer is really basic, it doesn't recognize verbs and nouns
sw = [x.replace('\n', '') for x in open('stopwords-fr.txt', mode="r", encoding="utf-8").readlines()] + stopwords.words('french') + stopwords.words('english')
lemmatizer = FrenchLefffLemmatizer()
stemmer = nltk.stem.snowball.FrenchStemmer()

#stem, lemmatize, remove punctuation, and return a string
def process(article):
    res = ""
    for word in word_tokenize(article):
        if not word in sw+string.punctuation.split():
            if res == "": #if first word
                res += stemmer.stem(lemmatizer.lemmatize(word.lower()))
            else:
                res += " " + stemmer.stem(lemmatizer.lemmatize(word.lower()))
    return res


In [3]:
data = pd.read_csv('export_articles_EGC_2004_2018.csv', sep='\t', header=0)

#Let's process our corpus, and determine a limit to split it in partitions
limit = 0
usable = []
for i in range(0, len(data['abstract']), 1):
    if not isinstance(data['abstract'][i], float): #if not null, empty, or whatever (so if there is a abstract)
        usable.append(process(data['abstract'][i]))
        if data['year'][i] <= 2010:
            limit += 1
    
print("nombre d'articles =", len(usable))
print("limit =", limit)

usable

nombre d'articles = 1096
limit = 553


["cet plateform object permettr citoyen d'analyserpar tweet polit d'éven spécif france.pour cas l'élect présidentiel 2017 , # idéo2017 analys quasitemp réel messag candidat , fourn principal caractérist , l'usag lexiqu polit comparaison candidat .",
 "la classif crois ( co-clustering ) techniqu d'extrair structuresous-jacent exist lign colon d'un tabl don form bloc . plusieur appliqu utilisent techniqu , algorithm co-clustering actuel passent l'échel . une approch utilis succes méthod modl , optimis criter vraisembl régularis . cependent , taill import , méthod atteint limit . dan articl , présenton nouvel algorithm co-clustering niveau , compt tenu criter modl trait efficac don grand taill , pouv mémoir . nos expérient montrent l'approch propos gagn temp calcul produis solut qualit .",
 "la recommand point d'intérêt devenu caractéristiqueessentiel réseau social géo-localis accompagnél'émergent échang massif don digital . cependantl faibl densit point d'intérêt visit utilis rendentl pr

In [ ]:
#TfIdf

vectorizer = TfidfVectorizer(stop_words=sw, use_idf=True, ngram_range=(1,3))
tfidf = vectorizer.fit_transform(usable)

tfidf

In [35]:
nb_concepts = 30

min_gram = 1
max_gram = 3

cleandocs = []
for t in usable:
    doc = []
    for n in range(min_gram, max_gram+1):
        for gram in ngrams(t.split(), n):
            doc.append(" ".join(gram))
    cleandocs.append(doc)

corpus = []
first = True
if first:
    dictionary = corpora.Dictionary(cleandocs)
    first = False
for doc in cleandocs:
    newVec = dictionary.doc2bow(doc)
    corpus.append(newVec)
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
lsi = models.LsiModel(corpus_tfidf, num_topics=nb_concepts, id2word=dictionary)
corpus_lsi = lsi[corpus_tfidf]

i=0
for doc in corpus_lsi:
    print("document number ", i)
    i+=1
    print(doc)

document number  0
[(0, 0.0260446318750886), (1, -0.008674165062032165), (2, -0.0019368492410148763), (3, -0.00947961404130463), (4, 0.019388613715007486), (5, -0.018473658708837104), (6, -0.026914338603328942), (7, -0.003377104588401564), (8, -0.010464715624196331), (9, -0.01379498131154898), (10, -0.016947742017422197), (11, 0.00045835508867071386), (12, 0.020743522938089176), (13, 0.008709704624884088), (14, 0.007801504894339375), (15, -0.007480451089592545), (16, -0.01522581923779812), (17, 0.03630749642348652), (18, 0.038590080506095484), (19, 0.0014331519450262286), (20, 0.011438935313976266), (21, 0.008829615514948739), (22, 0.04698574000039401), (23, -0.02430638443711234), (24, -0.035686096182014265), (25, 0.008459842571443254), (26, -0.004995646140413623), (27, -0.0362887528066963), (28, 0.01660366954330528), (29, -0.004752503060746873)]
document number  1
[(0, 0.09228739769845666), (1, 0.03409798166555594), (2, 0.013552293130687839), (3, -0.0772327567854726), (4, 0.0210722701

document number  313
[(0, 0.039231978451973674), (1, -0.126059470692322), (2, -0.017035956263982785), (3, -4.314413742747125e-05), (4, 0.0038831924929196827), (5, -0.01190055143944062), (6, -0.0010895969950393773), (7, 0.02604728212212787), (8, 0.006295879196885676), (9, 0.01894441760914401), (10, 0.03673369503461188), (11, 0.04945948199910629), (12, -0.014737110513967032), (13, 0.06973724389935923), (14, 0.03176437170959236), (15, -0.031814615731296424), (16, 0.0021227062353248653), (17, -0.02991375147124639), (18, 0.014175087403177402), (19, 0.021078767218599897), (20, 0.009215523215005735), (21, 0.009156366485218078), (22, 0.028193327320451395), (23, 0.0007132593841777723), (24, 0.057349994276861355), (25, -0.013307993086612757), (26, 0.03801392832956498), (27, -0.066313122599592), (28, 0.05244002468002397), (29, 0.03195114474856552)]
document number  314
[(0, 0.06408435612760355), (1, -0.0305041155756245), (2, 0.003906332719042262), (3, 0.038841629350735256), (4, 0.0346552566052631

document number  479
[(0, 0.0992381721563439), (1, 0.00843614077063562), (2, -0.011262291824461229), (3, 0.013376459814049821), (4, -0.08424705511190227), (5, 0.04393181126501769), (6, 0.029522347431685064), (7, 0.021837543758249953), (8, -0.013384184150091875), (9, 0.01040200618896156), (10, 0.006819177727273332), (11, 0.02313041534511465), (12, 0.024872460990533297), (13, 0.03440623577016272), (14, -0.013708647084775661), (15, 0.013282946303313304), (16, -0.000288264154662292), (17, 0.011341465279472386), (18, -0.023271915166574397), (19, -0.02316641186964172), (20, -0.0011921501381939484), (21, -0.014523248426101634), (22, 0.03683657650702736), (23, 0.0633599740766401), (24, -0.000350558609427306), (25, -0.016806550515050027), (26, 0.034072334324539634), (27, 0.02240217678774953), (28, 0.001733786708214367), (29, -0.034000679950779794)]
document number  480
[(0, 0.06917831751505359), (1, -0.013951208023510433), (2, -0.006011742118176227), (3, 0.003795595423639749), (4, 0.03682008711

document number  512
[(0, 0.029171091907519535), (1, -0.23108081982328804), (2, 0.011215296646134396), (3, -0.02834811930443643), (4, -0.02527461951180521), (5, -2.9684656430631296e-05), (6, -0.004985289662498337), (7, -0.0025428705434837546), (8, -0.015790138847177927), (9, 0.007405336512395189), (10, -0.03364517692845983), (11, -0.020222607263332488), (12, -0.018842851513364216), (13, -0.01542418751559678), (14, -0.006821649619876766), (15, 0.014064101088914737), (16, -0.009608087327960384), (17, 0.028550431522727654), (18, -0.042848223487073955), (19, -0.013099672952406159), (20, -0.028784118756011372), (21, 0.0031649847478580134), (22, -0.03589154898547786), (23, -0.012204404650684023), (24, -0.03308553616537325), (25, 0.03203033322043242), (26, -0.008276422028115012), (27, 0.01782644827932322), (28, -0.0010197907905164108), (29, 0.010088822973699615)]
document number  513
[(0, 0.07842245560987442), (1, 0.02605014388733477), (2, -0.026389875221972888), (3, 0.04657778109703105), (4,

document number  591
[(0, 0.05956742569491639), (1, -0.0039475591043373), (2, -0.026300846611124477), (3, 0.022309344352497495), (4, 0.020622785647113326), (5, 0.04748517127908022), (6, -0.006120806049114294), (7, -0.028028621990293743), (8, 0.0703117774946904), (9, 0.004790871969571645), (10, 0.030167852852997737), (11, 0.03045762406509552), (12, -0.003431421188270067), (13, 0.023739648444212787), (14, 0.03336597667594113), (15, 0.049889479322693164), (16, 0.006325310351099201), (17, -0.06066119585037508), (18, -0.0512827318930835), (19, 0.03024375175296551), (20, 0.011351184980542236), (21, -0.0043419530412296715), (22, -0.013675295966274434), (23, -0.0017444980687456421), (24, -0.04378654256347847), (25, -0.005610882005135971), (26, -0.01225704374514109), (27, 0.00586312301462155), (28, 0.030713693702000888), (29, -0.026436294017007336)]
document number  592
[(0, 0.05511145148839032), (1, 0.007613269331281709), (2, -0.002220401435832776), (3, 0.0392199432732885), (4, 0.0208496090711

...

document number  738
[(0, 0.07457781736155793), (1, -0.0053161422614012455), (2, 0.03653187494395497), (3, -0.02635850776589413), (4, -0.0049860521899318605), (5, -0.030894022642068055), (6, 0.03223236630497884), (7, 0.0478862588081705), (8, -0.0200669206298325), (9, -0.041434026042666744), (10, -0.003702401323185503), (11, 0.010406174513348335), (12, -0.029534652244623493), (13, -0.041510175414363304), (14, -0.06818700907028877), (15, 0.019816995984552273), (16, -0.02998402381627158), (17, -0.007028088292157393), (18, 0.0283234475267657), (19, 0.08111146377893381), (20, 0.04149571488768539), (21, 0.0612223170216786), (22, 0.019982964026486836), (23, 0.1543151895582743), (24, -0.027241361373843892), (25, -0.11826118961576434), (26, -0.02151728216235831), (27, 0.013803502852291361), (28, 0.12386022571155288), (29, -0.07934766131946203)]
document number  739
[(0, 0.07444714980722404), (1, 0.0018243466115863177), (2, 0.011323324167444755), (3, 0.0394857898957563), (4, 0.020494829425198927

document number  809
[(0, 0.12661188105585747), (1, 0.0002447722436833017), (2, -0.04791652235076091), (3, -0.07764952629698924), (4, 0.014344614269675211), (5, 0.03337191646141507), (6, 0.01736486668418727), (7, -0.050754530279400696), (8, -0.054601465165474664), (9, 0.03228676869953877), (10, 0.03369932019096324), (11, -0.0017230206560332052), (12, -0.01588471820051678), (13, 0.02453121429033927), (14, 0.011723788350669508), (15, -0.0047950066449003615), (16, -0.021365713822688353), (17, -0.025859134148266058), (18, -0.0376541963093061), (19, -0.0017306769783006488), (20, -0.031824295697919766), (21, -0.0022220832621182274), (22, 0.0042847628556611575), (23, -0.04113618063392401), (24, -0.003667156799867616), (25, 0.015287714774340034), (26, 0.02135119178254761), (27, 0.03782038090172194), (28, -0.02800068960821484), (29, 0.011833528872651655)]
document number  810
[(0, 0.05431155290590383), (1, -0.00018034885899253112), (2, -0.01146359426092599), (3, 0.0196166314536065), (4, -0.0013

document number  859
[(0, 0.11548145193882224), (1, 0.03608934850364764), (2, 0.1669220634473635), (3, 0.05575256787307973), (4, 0.0803436867081141), (5, 0.03716586257186471), (6, 0.02296951614120615), (7, -0.006800612620234723), (8, -0.0279583952981782), (9, 0.0013630962470794592), (10, 0.03026334613973048), (11, -0.02775698168475472), (12, -0.004405197350990238), (13, 0.008337890150222873), (14, 0.013022736467618933), (15, 0.02420125837811425), (16, 0.03969030104538332), (17, 0.011872322260340419), (18, 0.032948610484499534), (19, -0.00010557151318288243), (20, -0.008693403459663246), (21, 0.013219057023361605), (22, -0.030321244863500726), (23, -0.025510457920681417), (24, 0.011745557748221122), (25, 0.04818827255925039), (26, 0.038803770005031706), (27, 0.01976034811912002), (28, -0.009145761658841997), (29, -0.00255708711181471)]
document number  860
[(0, 0.07576638675934479), (1, -0.00046995042391973055), (2, 0.025088260160405162), (3, 0.020456349604192372), (4, -0.09169431987713

document number  1024
[(0, 0.07403197103965786), (1, 0.0022139649709743267), (2, -0.011989570724344607), (3, 0.038786348227225535), (4, 0.0022738422207065827), (5, -0.024396724710729272), (6, -0.010953255283133666), (7, 0.02294290941943565), (8, -0.009178271029203627), (9, -0.026508625957738636), (10, -0.01663497764889277), (11, 0.017359048651233652), (12, -0.01042040355479514), (13, 0.03985828821587481), (14, -0.011274337118393304), (15, 0.04433800352465411), (16, -0.013219603136010012), (17, -0.007550015161252134), (18, 0.07254927314110808), (19, -0.029554305751867124), (20, -0.014055657916887286), (21, 0.028645583900896495), (22, -0.01599152907462511), (23, -0.027490438786285404), (24, -0.06186182713543), (25, -0.036877275766244264), (26, 0.04607306178730711), (27, 0.03120815176526949), (28, -0.007932014061325257), (29, 0.03421494930495276)]
document number  1025
[(0, 0.077669443382372), (1, -0.004556726114835215), (2, 0.059669911083543956), (3, 0.030207130540580517), (4, -0.0071159

document number  1095
[(0, 0.08754817631528787), (1, 0.008014774687443126), (2, -0.005486420362632347), (3, 0.01648252172958719), (4, 0.047187989200138364), (5, -0.011898918734008652), (6, -0.006461471134603517), (7, 0.03368619256888739), (8, -0.015159611553418025), (9, -0.1149469341741261), (10, -0.00232096555072157), (11, -0.05373875693042852), (12, -0.02306555206382201), (13, -0.009932421911546769), (14, 0.025385789184931735), (15, 0.0038372191279384734), (16, 0.021963142785810012), (17, -0.005414913065323147), (18, 0.019685458003208436), (19, 0.05527415922491683), (20, -0.01645843140130053), (21, 0.010679290012169471), (22, 0.04232247644482326), (23, -0.006125946680772119), (24, -0.011023714642820754), (25, 0.004706630738879834), (26, 0.06874324816463104), (27, -0.027528666504666996), (28, -0.0029719048832206273), (29, 0.04252007506773781)]


In [41]:
clusters = []
tresh = 0.03

for i in range(0, nb_concepts):
    dic = {}
    num_doc = 0
    for doc in corpus_lsi:
        if abs(doc[i][1]) > tresh:
            dic[num_doc] = doc[i][1]
        num_doc+=1
    clusters.append(dic)
clusters

[{0: 0.0260446318750886,
  1: 0.09228739769845666,
  2: 0.07008707810255761,
  3: 0.06505391454501575,
  4: 0.0742512717357134,
  5: 0.058910826308553606,
  6: 0.07866496446405388,
  7: 0.05938727689399112,
  8: 0.05722778888846579,
  9: 0.05916010447671594,
  10: 0.10330950328299231,
  11: 0.06907823368218859,
  12: 0.07539327030644223,
  13: 0.08580215988782333,
  14: 0.08379386977353086,
  15: 0.07678224664166435,
  16: 0.11096433885028997,
  17: 0.07454971199293652,
  18: 0.08474448435677907,
  19: 0.0375636930426811,
  20: 0.05313909685659994,
  21: 0.06569816116784957,
  22: 0.07515221490501327,
  23: 0.057808149344063545,
  24: 0.05652073091179157,
  25: 0.0630091915045657,
  26: 0.061873484476506375,
  27: 0.06106702937767089,
  28: 0.039864649647146255,
  29: 0.07814083768950507,
  30: 0.0381691563268145,
  31: 0.04408905944049099,
  32: 0.06232376884297911,
  33: 0.07635344636511893,
  34: 0.07094604120113245,
  35: 0.06345939406888283,
  36: 0.0668578015355018,
  37: 0.04639

#### Nous obtenons ici x clusters, représentés par des concepts grâce au LSA
#### Nous allons seuiller et étiquetter ces clusters

In [ ]:
#Here, I determine a tresh, to remove documents from clusters
#Thus, I tried to compute the X² of the values for each cluster. This X² was my tresh
#Yeah yeah yeah. I know, X² is a law, what I compute is just a value to compare to this law
#But I find this more pictured to talk about it this way
#PS: (don't trust this snippet, I think it's wrong)

partitions = []
tresh = 0

for lsa_part in [lsa1, lsa2]:
    # X² pour ne garder que les articles au dessus d'un certain seuil
    temp = 0
    sumConcept = 0
    for concept in transpose(lsa):
        for article in concept:
            sumConcept += article
            
            
            tresh += pow(article, 2)
    tresh /= (lsa_part.shape[0]*lsa_part.shape[1])
    

In [ ]:
def indexOf(matrixSrc, listSearched):
    if (len(matrixSrc[1]) != len(listSearched)):
        return -1
    pos = 0
    for l in matrixSrc:
        thesame = True
        for i in range(0, len(listSearched), 1):
            if l[i] != listSearched[i]:
                thesame = False
                break
        if thesame:
            return pos
        pos+=1
    return -2
a = [[1, 2, 3], [4, 2, 1], [2, 3, 5], [3, 2, 1]]
b = [3,2,1]
indexOf(a, b)

In [ ]:
#I know, transpose exists in numpy
#However, it didn't work with me because I got sparse matrix, so I was deseperate and I wrote my own function
def transpose(matrix):
    transp = []
    for j in range(0, matrix.shape[1], 1):
        l = []
        for i in range(0, matrix.shape[0], 1):
            l.append(matrix[i][j])
        transp.append(l)
    return transp


In [ ]:
def mostCommons(dico, tresh):
    res = []
    for name in dico:
        if dico[name] >= tresh:
            res.append(name)
    return res

test = {"abc": 3, "trucmuch":5, "doge":0}
mostCommons(test, 3)

In [ ]:
#IMPORTANT!! The good way to get an element from document-term matrix
tfidf.toarray().item(0, 0)